In [ ]:
import time
from isimip_client.client import ISIMIPClient
client = ISIMIPClient()

In [ ]:
# mask a file for a country
# run this subsequently to poll the status
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2015_2020.nc'
response = client.mask(path, country='deu')
response

In [ ]:
# the same works for more then one file, e.g. a dataset, it just takes longer
paths = [
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2015_2020.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2021_2030.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2031_2040.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2041_2050.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2051_2060.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2061_2070.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2071_2080.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2081_2090.nc',
    'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2091_2100.nc'
]
response = client.mask(paths, country='deu')
response

In [ ]:
# to mask everything but a bounding box in lat/lon use
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2015_2020.nc'
response = client.mask(path, bbox=[0, 10, 0, 10]) # [south, north, west, east]
response

In [ ]:
# to mask out all sea and antarctica data use:
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2015_2020.nc'
response = client.mask(path, landonly=True)
response

In [ ]:
# once the status is 'finished', get the url to download the result
client.download(response['file_url'], path='downloads', validate=False)

In [ ]:
# To mask a specific dataset, the following code can be used to first search the
# repository and then mask every file in the dataset. After starting the masking job 
# (on the server) it checks every 10 seconds, if the job is done and then downloads
# the file. Masking the files on the server can take a few minutes or even longer.

# get the dataset metadata from the ISIMIP repository
response = client.datasets(simulation_round='ISIMIP3b',
                           product='InputData',
                           climate_forcing='gfdl-esm4',
                           climate_scenario='ssp126',
                           climate_variable='tas',
                           time_step = 'daily')

if not response['count'] == 1:
    raise RuntimeError('More than one dataset retrieved')

dataset = response["results"][0]
paths = [file['path'] for file in dataset['files']]

for i in range(60):
    # start/check a masking job on the server (e.g. for China)
    response = client.mask(paths=paths, country='chn')

    # check the status and download the file if it is ready
    print(response['status'], response['meta'])
    if response['status'] == 'finished':
        client.download(response['file_url'], path='downloads', validate=False)
        break
    else:
        time.sleep(10)